In [25]:
import langchain 
langchain.__version__

'0.3.20'

In [26]:
from langchain.agents import initialize_agent, Tool
# from langchain_community.llms import Ollama
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
import os


In [27]:
load_dotenv()

API_KEY = os.getenv("GOOGLE_AI_API")

# Search Web

In [28]:
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

search_tool = Tool(
    name = "DuckDuckGo Search",
    func= DuckDuckGoSearchRun().run,
    description="Use this tool to perform web searches using DuckDuckGo"
)

llm = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=API_KEY)

tool = [search_tool]

agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)

response = agent.run("What are the latest top running movies in 2025")



> Entering new AgentExecutor chain...
I need to find the top running movies of 2025. Since it is only November 2023, I will need to search for projected or expected top movies of 2025.

Action: DuckDuckGo Search
Action Input: "most anticipated movies 2025"
Observation: Find out what movies are coming in 2025, from superhero blockbusters to musical biopics, horror films to action thrillers. See the release dates, directors, stars, and plot summaries of the most noteworthy titles of the year. Find out what's coming to the big screen in 2025, from superhero sequels to musical adaptations to horror remakes. Rolling Stone's David Fear previews the most exciting films of the year, featuring stars like Tom Cruise, Cate Blanchett, Ariana Grande and more. Find out which movies ticket-buyers are looking forward to in 2025, from Captain America to Jurassic World. See the rankings, genres, performances and more in this article by Fandango staff. Collider staff picks the most exciting films comin

# Wikipedia Search

In [31]:
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

wiki_search_tool = Tool(
    name="Wikipedia Search",
    func=WikipediaAPIWrapper().run,
    description="Use this tool to search Wikipedia for information on any topic"
)

llm = GoogleGenerativeAI(model="gemini-2.0-flash",google_api_key=API_KEY)


tool = [wiki_search_tool]

agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)

response = agent.run("Who is Elon Musk? Tell me the latest news of him")

print(response)



> Entering new AgentExecutor chain...
I need to find information about Elon Musk and recent news related to him. I will start by searching Wikipedia for a general overview of who he is. Then, I will search for recent news articles.

Action: Wikipedia Search
Action Input: Elon Musk

/home/rohan/miniconda3/envs/dev/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/rohan/miniconda3/envs/dev/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman known for his key roles in Tesla, Inc., SpaceX, and Twitter (which he rebranded as X). Since 2025, he has been a senior advisor to United States president Donald Trump and de facto head of the Department of Government Efficiency (DOGE). Musk is the wealthiest person in the world; as of February 2025, Forbes estimates his net worth to be US$353 billion. 
Born to a prominent family in Pretoria, South Africa, Musk emigrated to Canada in 1989 and acquired its citizenship though his mother. He moved to the U.S. and graduated from the University of Pennsylvania before moving to California to pursue business ventures. In 1995, Musk co-founded the software company Zip2. After its sale in 1999, he co-founded X.com, an online payment company that later merged to form PayPal, which was acquired by eBay in 2002 for $1.5 billion. That year, Musk also became a U.S. citizen.
In 2002, Musk founded Spa

# SQLite database

In [1]:
import sqlite3

connection = sqlite3.connect("example_new.db")
cursor = connection.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
        id INTEGER PRIMARY KEY,
        product_name TEXT NOT NULL,
        stock_quantity INTEGER NOT NULL
)
""")

cursor.executemany("""
INSERT INTO inventory(product_name, stock_quantity) VALUES(?,?)
""", [
    ("Product A", 150),
    ("Product B", 250),
    ("Product C", 350),
    ("Product D", 450),
    ("Product E", 550),
])

connection.commit()
connection.close()

print("Database created and populated with sample data")

Database created and populated with sample data


In [1]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

In [ ]:
db = SQLDatabase.from_uri("sqlite:///example_new.db")

llm = GoogleGenerativeAI(model="gemini-2.0-flash",google_api_key=API_KEY)

toolkit = SQLDatabaseToolkit(db=db,llm=llm)

tools = toolkit.get_tools()

agent = initialize_agent(tools=tools, llm=llm, agent="zero-shot-react-description", verbose=True)

query1="How many units of Product A are in stock ?"
response1 = agent.run(query1)
print("Response:",response1)